### Calculate efficiency in unweighted network

In [27]:
import networkx as nx

def calculate_e_global(G):
    E = 0.0
    Eid = 0.0
    path_lengths = nx.shortest_path_length(G)
    for i in path_lengths.values():
        for j in i.values():
            if j != 0:
                E += 1.0 / j
    for x in G.nodes():
        for y in G.nodes():
            if x != y:
                Eid += 1
    if Eid != 0:
        E = E / Eid
    else:
        E = 0
    return E
    
def calculate_e_local(G):
    avg = 0.0
    for node in G:
        avg += calculate_e_global(G.subgraph(nx.bfs_successors(G,node)[node]))
    avg = avg/len(G)
    return avg

def calculate_city_subway_efficiency(cityName):
    with open("D:/Data/quanturb-metro-network-data-cities-2009/" + cityName + "-2009-adjacency.net") as f:
        data =  f.readlines()
    stopdict = {}
    G = nx.Graph()
    for i in data:
        if i.split(" ")[-1] == '0\n': # The stop lines are ended with 0
            stop_id = i.split(" ")[0]
            stop_name = i.split(" ")[1][1:-1]
            stop_lat = i.split(" ")[2]
            stop_lon = i.split(" ")[3]
            stopdict[stop_id] = [stop_name, stop_lat, stop_lon]
        if i.split(" ")[-1] == '1\n': # The edge lines are ended with 1
            start = i.split(" ")[0]
            end = i.split(" ")[1]
            G.add_edge(start, end)
    print cityName, " Eglob = ", calculate_e_global(G)
    print cityName, " Eloc = ",calculate_e_local(G)

In [30]:
calculate_city_subway_efficiency("HongKong")
calculate_city_subway_efficiency("Madrid")
calculate_city_subway_efficiency("Chicago")
calculate_city_subway_efficiency("Berlin")
calculate_city_subway_efficiency("Beijing")
calculate_city_subway_efficiency("Barcelona")
calculate_city_subway_efficiency("NewYork")
calculate_city_subway_efficiency("Shanghai")
calculate_city_subway_efficiency("Tokyo")
calculate_city_subway_efficiency("Paris")
calculate_city_subway_efficiency("Seoul")
calculate_city_subway_efficiency("Moscow")
calculate_city_subway_efficiency("Mexico")
calculate_city_subway_efficiency("Osaka")

HongKong  Eglob =  0.150245958251
HongKong  Eloc =  0.00609756097561
Madrid  Eglob =  0.120476760921
Madrid  Eloc =  0.00614035087719
Chicago  Eglob =  0.114162607041
Chicago  Eloc =  0.0182033096927
Berlin  Eglob =  0.109005791837
Berlin  Eloc =  0.00588235294118
Beijing  Eglob =  0.149773533382
Beijing  Eloc =  0.0201923076923
Barcelona  Eglob =  0.14413603838
Barcelona  Eloc =  0.008984375
NewYork  Eglob =  0.0732230925809
NewYork  Eloc =  0.017277026284
Shanghai  Eglob =  0.119685146129
Shanghai  Eloc =  0.00292792792793
Tokyo  Eglob =  0.137223621792
Tokyo  Eloc =  0.0236705434862
Paris  Eglob =  0.116207527235
Paris  Eloc =  0.0215320910973
Seoul  Eglob =  0.0762680146974
Seoul  Eloc =  0.00595238095238
Moscow  Eglob =  0.151920095643
Moscow  Eloc =  0.0195273631841
Mexico  Eglob =  0.134071376436
Mexico  Eloc =  0.00340136054422
Osaka  Eglob =  0.162369801757
Osaka  Eloc =  0.0


### Calculate efficiency in weighted network

In [20]:
from math import *
import networkx as nx
import string

def dis(lat1,lon1,lat2,lon2):
    R = 6373.0 # convert Earth's radius in kilometers
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

def calculate_e_global_with_weight(G, stopdict):
    E = 0.0
    Eid = 0.0
    path_lengths = nx.shortest_path_length(G)
    for i in path_lengths.values():
        for j in i.values():
            if j != 0:
                E += 1.0 / j
    for x in G.nodes():
        for y in G.nodes():
            if x != y:
                l = dis(string.atof(stopdict[x][1]), 
                        string.atof(stopdict[x][2]), 
                        string.atof(stopdict[y][1]), 
                        string.atof(stopdict[y][2]))
                if l != 0:
                    Eid += 1.0 / l
    if Eid != 0:
        E = E / Eid
    else:
        E = 0
    return E
    
def calculate_e_local_with_weight(G, stopdict):
    avg = 0.0
    for node in G:
        avg += calculate_e_global_with_weight(G.subgraph(nx.bfs_successors(G,node)[node]), stopdict)
    avg = avg/len(G)
    return avg

def calculate_city_subway_efficiency_with_weight(cityName):
    with open("D:/Data/quanturb-metro-network-data-cities-2009/" + cityName + "-2009-adjacency.net") as f:
        data =  f.readlines()
    stopdict = {}
    G = nx.Graph()
    for i in data:
        if i.split(" ")[-1] == '0\n': # The stop lines are ended with 0
            stop_id = i.split(" ")[0]
            stop_name = i.split(" ")[1][1:-1]
            stop_lat = i.split(" ")[2]
            stop_lon = i.split(" ")[3]
            stopdict[stop_id] = [stop_name, stop_lat, stop_lon]
        if i.split(" ")[-1] == '1\n': # The edge lines are ended with 1
            start = i.split(" ")[0]
            end = i.split(" ")[1]
            geodistance = dis(string.atof(stopdict[start][1]), 
                              string.atof(stopdict[start][2]), 
                              string.atof(stopdict[end][1]), 
                              string.atof(stopdict[end][2]))
            G.add_edge(start, end, weight = geodistance)
    print cityName, " Eglob = ", calculate_e_global_with_weight(G, stopdict)
    print cityName, " Eloc = ",calculate_e_local_with_weight(G, stopdict)

In [29]:
calculate_city_subway_efficiency_with_weight("NewYork")
calculate_city_subway_efficiency_with_weight("HongKong")
calculate_city_subway_efficiency_with_weight("Madrid")
calculate_city_subway_efficiency_with_weight("Chicago")
calculate_city_subway_efficiency_with_weight("Berlin")
calculate_city_subway_efficiency_with_weight("Beijing")
calculate_city_subway_efficiency_with_weight("Barcelona")
calculate_city_subway_efficiency_with_weight("Shanghai")
calculate_city_subway_efficiency_with_weight("Tokyo")
calculate_city_subway_efficiency_with_weight("Paris")
calculate_city_subway_efficiency_with_weight("Seoul")
calculate_city_subway_efficiency_with_weight("Moscow")
calculate_city_subway_efficiency_with_weight("Mexico")
calculate_city_subway_efficiency_with_weight("Osaka")

NewYork  Eglob =  0.348931275681
NewYork  Eloc =  0.0182486259874
HongKong  Eglob =  0.98551284002
HongKong  Eloc =  0.0186248641969
Madrid  Eglob =  0.52975053902
Madrid  Eloc =  0.00461160552901
Chicago  Eglob =  0.646182853973
Chicago  Eloc =  0.023431476511
Berlin  Eglob =  0.560973532469
Berlin  Eloc =  0.0057605422705
Beijing  Eglob =  1.10916189678
Beijing  Eloc =  0.338246727988
Barcelona  Eglob =  0.489895893416
Barcelona  Eloc =  0.00746752754019
Shanghai  Eglob =  0.98365528006
Shanghai  Eloc =  0.00521446723422
Tokyo  Eglob =  0.685759683165
Tokyo  Eloc =  0.0267437729595
Paris  Eglob =  0.525752542603
Paris  Eloc =  0.0196056620361
Seoul  Eglob =  0.916725061182
Seoul  Eloc =  0.00671963241211
Moscow  Eglob =  1.20929758728
Moscow  Eloc =  0.0390569228812
Mexico  Eglob =  0.756009605063
Mexico  Eloc =  0.00408524485823
Osaka  Eglob =  0.761907483362
Osaka  Eloc =  0.0
